In [1]:
import datetime
import numpy as np 
import time
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pickle
import string
import pandas as pd
from copy import deepcopy

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# aux functions

In [3]:
def get_pe_ratio(soup_tag):
    
    pe_ratio = float(str(soup_tag[0]).split("<")[1].split(">")[-1])
    print("P/E ratio", pe_ratio)
    
    return pe_ratio
    
def get_short_float(soup_tag):
    
    short_float = float(str(soup_tag[-1]).split("<")[1].split(">")[-1][:-1])
    print("Short interest", short_float)
    
    return short_float

def get_prices(list_tag):
    
    median_estimate = float(list_tag[2] + "." + list_tag[3])
    high_estimate = float(list_tag[4] + "." + list_tag[5])
    low_estimate = float(list_tag[6] + "." + list_tag[7])
    current = float(list_tag[-2] + "." + list_tag[-1])
    
    median_change = ((median_estimate-current)/current)*100
    high_change = ((high_estimate-current)/current)*100
    low_change = ((low_estimate-current)/current)*100
    
    return median_change, high_change, low_change, current

# fetch company names and price

In [4]:
def get_nyse_stocks():
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    list_stock = []
    for letter in string.ascii_uppercase:
        print("Fetching for letter: " + letter)
        
        r = requests.get("https://eoddata.com/stocklist/NYSE/" + letter + ".htm", headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag = soup.find_all("a", href=True)

        for link in soup_tag:
            text = link.get('href')
            if "/stockquote/NYSE/" in text:
                text = text.split("/")[-1].split(".")[0]

                if text not in list_stock:
                    list_stock.append(text)
                    
    return list_stock

In [5]:
def get_interesting_stocks(list_stock):
    
    interesting_stocks = dict()
    database = dict()
    
    for symbol in list_stock:
        
        print(symbol)

        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        r = requests.get("https://money.cnn.com/quote/forecast/forecast.html?symb=" + symbol, headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        
        # number of analyst
        
        full_data = True
        
        try:
            soup_tag_research = soup.find_all("div", attrs={"class": "wsod_twoCol clearfix"})
            price_strings = re.findall(r'\d+', str(soup_tag_research[0]).split("<p>")[1].split("</p>")[0])
        
            num_analysts = int(re.findall(r'\d+', str(soup_tag_research[1]).split("<p>")[1])[0])
            median_change, high_change, low_change, current = get_prices(price_strings)
            
            if low_change > 0 and num_analysts >= 6:
                interesting_stocks[symbol] = low_change
            
        except IndexError: 
            print("No data found for symbol: " + symbol)
            full_data = False
            
        if full_data:
            
            r = requests.get("https://www.wsj.com/market-data/quotes/" + symbol, headers=headers)
            html_doc = r.text
            soup = BeautifulSoup(html_doc, 'html.parser')
            soup_tag_overview = soup.find_all("span", attrs={"class": "WSJTheme--data_data--3CZkJ3RI"})

            try:
                short_float = get_short_float(soup_tag_overview)
            except (ValueError, IndexError):
                full_data = False
                print("No short float data found")
                
            try:
                pe_ratio = get_pe_ratio(soup_tag_overview)
            except (ValueError, IndexError):
                if full_data:
                    pe_ratio = 0.0
                else:
                    full_data = False
                print("No P/E data found")
                
        if full_data:
            database[symbol] = [pe_ratio, short_float, median_change, high_change, low_change, current, num_analysts]
            
            

    return interesting_stocks, database

In [30]:
def scrap_zacks(interesting_stocks, database, symbol):
    
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
        r = requests.get("https://www.zacks.com/stock/quote/" + symbol + "?q=aapl", headers=headers, timeout=60)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        soup_tag_rank = soup.find_all("p",  attrs={"class": "rank_view"})
        soup_tag = soup.find_all("span", attrs={"class": "composite_val"})
        
        try:
            if len(soup_tag) == 8:
                soup_tag = soup_tag[-4:-1]
                score_list = [char for tag in soup_tag for char in str(tag) if char.isupper()]
                
                if score_list:
                    score_list.sort()
                    if score_list[-1] <= 'B' and symbol in interesting_stocks.keys():
                        potential_upside = interesting_stocks[symbol]
                        interesting_stocks[symbol] = [potential_upside, score_list]
                        
                buy_reccomendation = re.findall(r'\d+', str(soup_tag_rank).split("<")[1])
                database[symbol] = database[symbol] + [score_list, buy_reccomendation]
                print(symbol, database[symbol])
                
            else:
                print("No data found for symbol: " + symbol)
                
        except IndexError:
            print("No data found for symbol: " + symbol)
            
        return interesting_stocks, database

In [31]:
def get_interesting_stocks_zacks(interesting_stocks, database):
    
    for symbol in database.keys():
        time.sleep(1)
        
        try:
            interesting_stocks, database = scrap_zacks(interesting_stocks, database, symbol)    
        except (requests.exceptions.RequestException, ValueError) as e:
            print("caught", e, symbol)
            interesting_stocks, database = scrap_zacks(interesting_stocks, database, symbol)
    
    return interesting_stocks, database
            

In [7]:
list_stock = get_nyse_stocks()

Fetching for letter: A
Fetching for letter: B
Fetching for letter: C
Fetching for letter: D
Fetching for letter: E
Fetching for letter: F
Fetching for letter: G
Fetching for letter: H
Fetching for letter: I
Fetching for letter: J
Fetching for letter: K
Fetching for letter: L
Fetching for letter: M
Fetching for letter: N
Fetching for letter: O
Fetching for letter: P
Fetching for letter: Q
Fetching for letter: R
Fetching for letter: S
Fetching for letter: T
Fetching for letter: U
Fetching for letter: V
Fetching for letter: W
Fetching for letter: X
Fetching for letter: Y
Fetching for letter: Z


In [25]:
#interesting_stocks, database = get_interesting_stocks(list_stock)

A
Short interest 0.72
P/E ratio 45.65
AA


KeyboardInterrupt: 

In [32]:
interesting_stocks_zacks, database_zacks = get_interesting_stocks_zacks(deepcopy(interesting_stocks), deepcopy(database))

A [45.65, 0.72, 8.520310896025814, 17.31925502273061, -1.0118785745710481, 136.38, 18, ['A', 'B', 'F'], ['2']]
AA [0.0, 5.49, 1.0362694300518096, 45.07772020725388, -30.05181347150259, 38.6, 13, ['B', 'B', 'D'], ['1']]
AAIC [8.17, 3.7, 12.219451371571079, 24.68827930174564, -0.24937655860348598, 4.01, 3, ['B', 'F', 'F'], ['4']]
AAN [0.0, 2.33, 2.521474092546402, 16.375727348295914, -14.103629814353013, 36.09, 10, ['A', 'B', 'D'], ['1']]
AAP [20.48, 5.05, 17.622458047989966, 28.077787652255743, -28.90375869099273, 191.29, 25, ['B', 'B', 'B'], ['2']]
AAT [135.57, 2.04, -9.685863874345555, 9.947643979057585, -18.84816753926702, 38.2, 4, ['A', 'C', 'C'], ['3']]
AB [15.25, 1.11, -2.848310885405165, 1.567674983440055, -16.096268491940823, 45.29, 9, ['A', 'B', 'C'], ['3']]
ABB [13.75, 0.27, -4.822408316488598, 24.7473289055732, -36.76003465203582, 34.63, 33, ['C', 'D', 'F'], ['3']]
ABBV [39.62, 0.87, 11.833897700944576, 28.31937266084477, -6.433790768134022, 112.22, 23, ['A', 'A', 'A'], ['3']

ALX [25.91, 1.98, 7.307651035518835, 7.307651035518835, 7.307651035518835, 279.57, 1, ['B', 'C', 'F'], ['3']]
AM [13.49, 2.66, -9.54773869346733, 10.552763819095485, -24.623115577889443, 9.95, 12, ['B', 'C', 'C'], ['5']]
No data found for symbol: AMBC
AMC [0.0, 18.97, -92.15270413573701, -66.06574761399789, -97.87910922587487, 47.15, 10, ['F', 'F', 'F'], ['3']]
AMCR [21.67, 3.03, 6.616415410385268, 23.366834170854283, -3.6850921273031787, 11.94, 17, ['B', 'C', 'D'], ['3']]
AME [39.07, 0.56, 10.17754073208154, 24.20545042741287, 0.09498063856213593, 136.87, 18, ['B', 'B', 'C'], ['3']]
AMG [20.63, 2.32, -2.4522831601836206, 28.654264315051943, -10.606426673109448, 165.56, 10, ['A', 'C', 'D'], ['3']]
AMH [125.86, 1.51, -88.54302903478943, -98.84907140988753, -99.13680355741565, 38.23, 17, ['B', 'C', 'D'], ['3']]
AMK [0.0, 1.38, 11.581377452866494, 30.819545979222784, -7.6567910734897975, 25.99, 7, ['B', 'D', 'D'], ['3']]
AMN [34.77, 1.88, 12.951807228915671, 18.33046471600689, -1.03270223

AVYA [0.0, 14.64, 21.515368120085775, 42.95925661186561, -7.076483202287349, 27.98, 10, ['A', 'B', 'C'], ['4']]
AWI [29.07, 2.91, -6.902388564980255, 8.143690050780512, -11.604288132405495, 106.34, 9, ['C', 'D', 'F'], ['3']]
AWK [39.46, 1.16, 5.148425979355005, 16.04795794062961, -6.392254920818105, 155.97, 18, ['D', 'D', 'D'], ['3']]
AWR [32.14, 2.01, 8.271433967014973, 18.34319526627218, -11.872088631499441, 79.43, 4, ['C', 'C', 'C'], ['3']]
AX [13.88, 3.18, 13.42155009451796, 26.0239445494644, 5.019953791220333, 47.61, 7, ['B', 'D', 'F'], ['3']]
AXL [0.0, 5.9, -6.759906759906754, 16.550116550116556, -45.609945609945605, 12.87, 10, ['A', 'A', 'F'], ['1']]
AXP [26.99, 0.84, -6.394359348407494, 16.095307561390708, -27.06053975200584, 164.52, 27, ['C', 'C', 'F'], ['3']]
No data found for symbol: AXR
AXS [29.92, 1.09, 11.467976572832043, 22.803703003967506, -14.982051766484036, 52.93, 9, ['B', 'B', 'C'], ['3']]
AXTA [92.1, 3.14, 15.995115995116002, 40.41514041514042, 6.8376068376068435, 

BLU [0.0, 3.85, 143.55300859598853, 243.83954154727792, -25.50143266475645, 3.49, 8, ['B', 'F', 'F'], ['4']]
BMA [3.83, 0.34, 4.16666666666666, 221.29629629629628, -61.51620370370371, 17.28, 10, ['C', 'C', 'C'], ['5']]
BMI [52.76, 4.4, 0.08656135035706372, 2.791603549015363, -1.5364639688379156, 92.42, 7, ['A', 'B', 'D'], ['3']]
BMO [15.22, 0.5, 17.59172154280339, 20.41392285983067, 12.888052681091256, 106.3, 7, ['A', 'B', 'B'], ['2']]
BMY [0.0, 0.72, 15.740740740740748, 38.88888888888889, -4.320987654320983, 64.8, 19, ['A', 'A', 'D'], ['3']]
No data found for symbol: BNED
BNL [53.64, 0.42, -2.4183283835383973, -2.4183283835383973, -10.90369113279593, 23.57, 7, ['B', 'C', 'F'], ['4']]
BNS [14.14, 0.49, 17.106433442039737, 17.106433442039737, 17.106433442039737, 67.46, 5, ['A', 'F', 'F'], ['3']]
BOH [19.82, 3.93, 7.078449053201083, 12.714156898106404, -3.065825067628493, 88.72, 7, ['C', 'D', 'D'], ['2']]
BOOT [37.86, 6.72, 10.245576973857942, 34.67124372854503, -10.219170847636645, 75.7

CDAY [0.0, 6.35, 26.253687315634217, 41.5929203539823, 0.2949852507374631, 84.75, 17, ['C', 'F', 'F'], ['5']]
CDE [60.2, 1.98, 4.912998976458551, 43.295803480040945, 2.354145342886391, 9.77, 9, ['B', 'B', 'C'], ['4']]
CDR [0.0, 1.26, 22.28260869565217, 29.076086956521735, 8.69565217391304, 14.72, 4, ['A', 'D', 'D'], ['5']]
CE [9.3, 1.04, 8.26006340092111, 19.624379448531613, -8.487349721873317, 167.19, 23, ['A', 'C', 'D'], ['3']]
CEIX [61.03, 4.98, -16.61703394877903, -10.661107802263247, -16.61703394877903, 16.79, 3, ['A', 'A', 'B'], ['3']]
No data found for symbol: CELP
CEQP [0.0, 4.51, 2.564102564102564, 19.65811965811966, -7.6923076923076925, 29.25, 9, ['A', 'A', 'B'], ['3']]
CF [29.61, 1.07, 9.011627906976747, 18.095930232558143, -9.156976744186045, 55.04, 19, ['B', 'B', 'C'], ['1']]
CFG [14.11, 1.62, 2.3653349234744536, 13.29755515802027, -10.554561717352419, 50.31, 20, ['B', 'C', 'F'], ['3']]
CFR [19.9, 3.71, -9.487369373817165, 18.48926191063935, -21.830000822842095, 121.53, 16

...

SITE [49.77, 7.35, 20.176339150914906, 38.897276405580044, -20.285041367232324, 165.59, 8, ['B', 'F', 'F'], ['1']]
SIX [0.0, 4.95, 18.81615899762368, 57.70144739684597, -22.22942320155541, 46.29, 14, ['A', 'D', 'F'], ['3']]
SJI [15.18, 5.98, -0.21770682148040174, 30.624092888243837, -5.660377358490562, 27.56, 11, ['A', 'B', 'F'], ['3']]
SJM [16.42, 11.4, -10.494274532540942, 0.014494854326721433, -32.59892738077982, 137.98, 17, ['A', 'B', 'B'], ['3']]
SJR [27.69, 0.24, 8.456375838926173, 12.651006711409392, -11.81208053691275, 29.8, 11, ['C', 'C', 'C'], ['3']]
No data found for symbol: SJT
SJW [29.87, 1.08, 4.328869511055747, 37.028962939894114, -3.4568670196200544, 64.22, 6, ['B', 'C', 'C'], ['4']]
SKLZ [0.0, 30.88, 14.231318419800088, 61.827701094716794, -28.605425987624944, 21.01, 6, ['F', 'F', 'F'], ['3']]
SKM [16.86, 0.32, 19.113573407202217, 44.783010156971365, -3.1086488150200107, 32.49, 25, ['B', 'B', 'D'], ['2']]
SKT [0.0, 21.04, -17.72823779193206, -9.766454352441613, -36.305

SUI [106.54, 3.77, 2.8504260946223883, 12.841610343814278, -5.377607992947402, 170.15, 8, ['B', 'C', 'D'], ['3']]
SUM [25.01, 6.22, 4.107244723331425, 19.79463776383342, -54.363947518539646, 35.06, 18, ['B', 'B', 'C'], ['3']]
SUN [7.39, 2.79, -2.0940984498232336, 3.3451183029643645, -15.69214033179223, 36.77, 9, ['A', 'A', 'B'], ['3']]
No data found for symbol: SUP
SWCH [83.53, 8.55, 5.633802816901413, 30.78470824949699, -9.456740442655931, 19.88, 14, ['A', 'A', 'D'], ['3']]
SWI [34.96, 9.52, 21.06537530266344, 39.22518159806296, -3.1476997578692467, 16.52, 9, ['B', 'B', 'C'], ['3']]
SWK [21.62, 1.61, 13.55069732378439, 25.8009800226159, -10.478703354692804, 212.24, 20, ['B', 'C', 'C'], ['3']]
No data found for symbol: SWM
SWN [0.0, 7.23, -0.7220216606498201, 44.4043321299639, -45.84837545126354, 5.54, 21, ['A', 'A', 'C'], ['3']]
SWX [13.8, 1.27, 13.449768622182424, 34.34841021047919, 0.014927601134505324, 66.99, 8, ['A', 'C', 'D'], ['3']]
SXC [42.4, 3.32, 9.536082474226808, 15.9793814

TPR [35.5, 3.07, 29.411764705882355, 57.647058823529406, -5.88235294117647, 42.5, 27, ['B', 'C', 'C'], ['2']]
TPX [19.14, 3.33, 21.276034800949116, 60.822567888215126, 5.4574215660427114, 37.93, 11, ['A', 'A', 'B'], ['2']]
No data found for symbol: TR
No data found for symbol: TREC
TREX [60.45, 6.21, 21.500264131008972, 32.06550449022714, 3.539355520338081, 94.65, 15, ['A', 'F', 'F'], ['2']]
TRGP [50.42, 1.38, 9.276347741622146, 31.13161728994658, -12.578921806702281, 41.18, 23, ['B', 'B', 'C'], ['1']]
TRI [8.17, 0.65, 3.9755351681957247, 22.324159021406732, -20.489296636085623, 98.1, 17, ['D', 'F', 'F'], ['2']]
TRN [0.0, 7.79, 12.834978843441469, 23.413258110014105, -1.2693935119887145, 28.36, 6, ['B', 'D', 'F'], ['3']]
TRNO [53.71, 1.51, 6.596632164375089, 11.231268345434877, -1.1277614707245542, 64.73, 11, ['A', 'C', 'D'], ['3']]
TROX [3.62, 3.5, 17.3512154233026, 34.115674769488685, -32.942162615255654, 23.86, 8, ['B', 'D', 'D'], ['1']]
TRP [29.63, 0.84, 9.658103148541626, 21.71141

V [48.61, 1.56, 16.166929686130104, 30.194634402945812, 2.577590741714885, 228.12, 39, ['D', 'F', 'F'], ['3']]
VAC [0.0, 6.42, 15.591397849462362, 28.434886499402623, 0.3584229390680969, 167.4, 6, ['B', 'B', 'F'], ['3']]
VALE [11.09, 1.25, 11.557340473003123, 33.86880856760375, -18.34002677376171, 22.41, 22, ['A', 'C', 'C'], ['2']]
VAPO [0.0, 10.75, 113.03901437371661, 131.00616016427105, 64.27104722792608, 19.48, 5, ['C', 'D', 'F'], ['3']]
VCRA [0.0, 11.54, 37.8299120234604, 64.22287390029325, -6.158357771261001, 34.1, 12, ['A', 'D', 'D'], ['3']]
VEC [14.55, 1.94, 27.356598475732053, 30.36502206177297, 24.348174889691137, 49.86, 2, ['A', 'B', 'D'], ['2']]
No data found for symbol: VEDL
VEEV [112.86, 1.28, 19.19786846164633, 26.209507782919644, -5.342869162810268, 285.24, 24, ['A', 'F', 'F'], ['2']]
VEI [0.0, 2.31, 10.605074821080033, 49.642160052049455, -8.91346779440468, 15.37, 7, ['B', 'B', 'D'], ['3']]
VEL [0.0, 0.7, 9.837631327602669, 14.613180515759305, 5.06208213944603, 10.47, 4

No data found for symbol: WPF
WPG [0.0, 31.77, -28.571428571428577, -28.571428571428577, -28.571428571428577, 4.2, 2, ['A', 'F', 'F'], ['2']]
WPM [37.12, 1.46, 22.03676315233467, 56.45467990703571, -15.487006127192052, 47.33, 18, ['A', 'C', 'D'], ['3']]
WPP [0.0, 0.11, -1.1656005685856337, 27.988628287135768, -35.57924662402274, 70.35, 22, ['A', 'A', 'F'], ['3']]
WRB [18.92, 1.2, 16.50485436893204, 19.093851132686083, 0.9708737864077669, 77.25, 10, ['C', 'D', 'F'], ['2']]
WRE [0.0, 3.17, -1.1299435028248632, 0.8878127522195273, -3.147699757869254, 24.78, 4, ['B', 'C', 'D'], ['3']]
WRI [49.66, 1.52, -11.712772218952317, 0.05885815185404099, -20.541494997057086, 33.98, 10, ['B', 'C', 'F'], ['3']]
WRK [0.0, 1.22, 9.61930659415363, 27.464309993201898, -16.723317471108096, 58.84, 16, ['A', 'D', 'F'], ['3']]
WSM [14.85, 8.36, 11.226231106777174, 31.033642125792287, -29.91223793271575, 164.08, 26, ['A', 'A', 'A'], ['2']]
WSO [38.32, 4.45, 4.496847678428376, 7.631753108781228, -15.357553380473

In [33]:
interesting_stocks_filtered = []

for k in interesting_stocks_zacks.keys():
    if isinstance(interesting_stocks_zacks[k], list):
        interesting_stocks_filtered.append((interesting_stocks_zacks[k], k))

In [34]:
print(len(interesting_stocks_filtered), len(database_zacks))

39 1803


# dump stocks into the database

In [13]:
# load the current database
data_list = pickle.load(open("stocks_database.p", "rb"))

In [14]:
# append the new data for the new trading day
data_list.append(database_zacks)

NameError: name 'database_zacks' is not defined

In [36]:
# dump the new data
pickle.dump(data_list, open("stocks_database.p", "wb"))

# print the interesting stocks

In [37]:
print(interesting_stocks_filtered)

[([9.605420486249502, ['A', 'B', 'B']], 'AES'), ([9.835247129306035, ['A', 'B', 'B']], 'ASGN'), ([79.24818191129776, ['A', 'A', 'B']], 'AZO'), ([15.26370939573874, ['A', 'B', 'B']], 'BALY'), ([14.170584284754865, ['A', 'A', 'B']], 'BBU'), ([12.888052681091256, ['A', 'B', 'B']], 'BMO'), ([1.6949152542372818, ['A', 'A', 'A']], 'CRK'), ([16.930572472594406, ['A', 'A', 'A']], 'CWH'), ([0.1408230323893028, ['A', 'A', 'B']], 'DCI'), ([6.408853216587615, ['A', 'B', 'B']], 'DECK'), ([1.4594552407710955, ['A', 'A', 'B']], 'DGX'), ([18.764845605700717, ['B', 'B', 'B']], 'DOOR'), ([3.263114415530769, ['A', 'B', 'B']], 'EPD'), ([12.98230654444682, ['A', 'B', 'B']], 'FNF'), ([0.8130081300813075, ['A', 'A', 'B']], 'HBM'), ([13.729414975889368, ['A', 'A', 'B']], 'HRC'), ([21.509273076103185, ['A', 'A', 'A']], 'HZO'), ([2.1375171106665274, ['A', 'A', 'B']], 'LEN'), ([4.079058031959625, ['A', 'B', 'B']], 'LOW'), ([6.301032198428599, ['A', 'A', 'A']], 'LPX'), ([5.70279467876663, ['A', 'A', 'B']], 'MCK')

In [38]:
interesting_stocks_filtered = sorted(interesting_stocks_filtered, key=lambda tup: tup[0])

In [39]:
interesting_stocks_filtered

[([0.08263703975760114, ['B', 'B', 'B']], 'RSG'),
 ([0.1408230323893028, ['A', 'A', 'B']], 'DCI'),
 ([0.18401144960131552, ['A', 'B', 'B']], 'PNM'),
 ([0.5193499748701663, ['A', 'B', 'B']], 'ST'),
 ([0.7474813129671772, ['B', 'B', 'B']], 'MDT'),
 ([0.8130081300813075, ['A', 'A', 'B']], 'HBM'),
 ([1.4594552407710955, ['A', 'A', 'B']], 'DGX'),
 ([1.5572105619498933, ['A', 'B', 'B']], 'RJF'),
 ([1.6949152542372818, ['A', 'A', 'A']], 'CRK'),
 ([2.1375171106665274, ['A', 'A', 'B']], 'LEN'),
 ([2.631578947368421, ['A', 'A', 'B']], 'WNS'),
 ([3.263114415530769, ['A', 'B', 'B']], 'EPD'),
 ([3.627009646302242, ['A', 'A', 'A']], 'PKX'),
 ([4.079058031959625, ['A', 'B', 'B']], 'LOW'),
 ([5.4574215660427114, ['A', 'A', 'B']], 'TPX'),
 ([5.596620908130933, ['A', 'A', 'B']], 'TGNA'),
 ([5.70279467876663, ['A', 'A', 'B']], 'MCK'),
 ([5.88235294117647, ['A', 'A', 'A']], 'VNT'),
 ([6.084243369734798, ['A', 'A', 'B']], 'TNL'),
 ([6.301032198428599, ['A', 'A', 'A']], 'LPX'),
 ([6.408853216587615, ['A', '